In [86]:
import numpy as np
import scipy as sp

import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import path

import torch
from torch.utils import data
from torch import nn
import torch.nn.functional as F
from torch.optim import SGD

In [94]:
class SynteticSet(data.Dataset):
    def __init__(self, df, features, target="target"):
        target_dict = {"Class_1": 0, "Class_2": 1, "Class_3": 2, "Class_4": 3}
        
        for i in target_dict:
            df[target][df[target] == i] = target_dict[i]
           
        prefix = "feature_{}"
        
        for i,feature in enumerate(features):
            features[i] = prefix.format(str(feature))
        
            
        self.X = torch.tensor(df[features].to_numpy()).float()
        self.Y = torch.tensor(df[target].to_numpy(dtype=np.int64))
        
        
        
    def __len__(self):
        return self.Y.shape[0]
    
    def __getitem__(self, i):
        return self.X[i], self.Y[i]
    

In [95]:
data_path = path.Path(".\\data")
df = pd.read_csv(data_path/"train.csv")
    
dataset = SynteticSet(df, [0, 1])
dataset_len = len(dataset)

trainset, evalset = data.random_split(dataset, 
                                      [int(dataset_len*0.7), int(dataset_len*0.3)])

trainloader = data.DataLoader(trainset, batch_size=128, shuffle=True)
evalloader = data.DataLoader(evalset, batch_size=128)


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [96]:
class NN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
                nn.Linear(2,4),
                nn.Softmax(dim=1)
            )
    def forward(self, x):
        return self.net(x)

In [97]:
net = NN()
lossF = nn.CrossEntropyLoss()

opt = SGD(net.parameters() ,lr=0.03)

In [108]:
def train_step():
    loss_list = list()
    for x,y in trainloader:

        opt.zero_grad()
        result = net(x)
        loss = lossF(result, y)

        loss.backward()
        opt.step()

        loss_list.append(loss.item())

    print(np.array(loss_list).mean())

In [109]:
for _ in range(10):
    train_step()

1.1744434162293318
1.173883053240872
1.1734220848658838
1.1730472832318852
1.172752327413402
1.172506103349996
1.1722534143074774
1.1720735486290355
1.1719193131657797
1.1717592828670411
